# Epitome of Salt: Sunset Show Notes 
## Author: Kevin Stull
We are going to see how we did at rating the LCI cards before they came out on arena. 

## DataSet
https://www.17lands.com/tier_list/b8dad7059ff24da28da636a1c50da7e8

Last updated: 2023-12-21 14:19

In [17]:
import pandas as pd
sl = pd.read_csv('Data/LCI_17lands.csv')
sl = sl[['Name', 'Tier']]
eos = pd.read_csv('Data/LCI_EoS.csv')
eos['Name'] = eos['Name'].str[2:]
eos['Name'] = eos['Name'].str.replace(r'\(.*', '', regex=True)
eos['Name'] = eos['Name'].str.strip()
sl['Name'] = sl['Name'].str.strip()
merged_df = pd.merge(sl, eos, on='Name', how='inner')
num_common_names = len(merged_df)
merged_left_df = pd.merge(eos, sl, on='Name', how='left', indicator=True)
unmatched_eos = merged_left_df[merged_left_df['_merge'] == 'left_only']
if len(unmatched_eos != 0):
    print('Error, Missing Cards!')
merged_df

Name Tier Jon Kevin Nate
0       Dusk Rose Reliquary   B-  C+    C+   C+
1       Deathcap Marionette   C+  C+    C-   C-
2    Deep Goblin Skulltaker   D-  C+    D+   D+
3           Deep-Cavern Bat   A-   B     B    B
4               Defossilize   C+  C-     D   D+
..                      ...  ...  ..   ...  ...
195        Bloodthorn Flail   D-  D-    D+   D+
196        Broodrage Mycoid   D-   D    D-    D
197      Canonized in Blood    F  C-    D-    G
198         Chupacabra Echo   B+  B+     B    B
199             Dead Weight   C+   B    C+   C+

[200 rows x 5 columns]

## Point System
This is how we score the distance between "correct" predictions and our predictions 

In [18]:
tier_num = {'F': 0., 'G':0., 'G+':0., 'D-': 1., 'D': 2., 'D+': 3., 'C-': 4., 'C': 5., 'C+': 6., 'B-': 7., 'B': 8., 'B+': 9., 'A-': 10., 'A': 11., 'A+': 12.}
raters = ['Tier', 'Jon', 'Kevin', 'Nate']
for rater in raters:
    merged_df[str(rater) + '_num'] = merged_df[rater].map(tier_num)
clean = merged_df.dropna().copy()

for rater in raters: 
    if rater != 'Tier':
        clean[str(rater) + '_error'] = abs(clean['Tier_num'] - clean[str(rater) + '_num'])

clean

Name Tier Jon Kevin Nate  Tier_num  Jon_num  Kevin_num  \
0       Dusk Rose Reliquary   B-  C+    C+   C+       7.0      6.0        6.0   
1       Deathcap Marionette   C+  C+    C-   C-       6.0      6.0        4.0   
2    Deep Goblin Skulltaker   D-  C+    D+   D+       1.0      6.0        3.0   
3           Deep-Cavern Bat   A-   B     B    B      10.0      8.0        8.0   
4               Defossilize   C+  C-     D   D+       6.0      4.0        2.0   
..                      ...  ...  ..   ...  ...       ...      ...        ...   
195        Bloodthorn Flail   D-  D-    D+   D+       1.0      1.0        3.0   
196        Broodrage Mycoid   D-   D    D-    D       1.0      2.0        1.0   
197      Canonized in Blood    F  C-    D-    G       0.0      4.0        1.0   
198         Chupacabra Echo   B+  B+     B    B       9.0      9.0        8.0   
199             Dead Weight   C+   B    C+   C+       6.0      8.0        6.0   

     Nate_num  Jon_error  Kevin_error  Nate_error  
0         6.0        1.0          1.0         1.0  
1         4.0        0.0          2.0         2.0  
2         3.0        5.0          2.0         2.0  
3         8.0        2.0          2.0         2.0  
4         3.0        2.0          4.0         3.0  
..        ...        ...          ...         ...  
195       3.0        0.0          2.0         2.0  
196       2.0        1.0          0.0         1.0  
197       0.0        4.0          1.0         0.0  
198       8.0        0.0          1.0         1.0  
199       6.0        2.0          0.0         0.0  

[147 rows x 12 columns]

## And The Winner Is... Nate!!!

In [19]:
print('Error for Jon:', clean['Jon_error'].sum())
print('Error for Kevin:', clean['Kevin_error'].sum())
print('Error for Nate:', clean['Nate_error'].sum())

Error for Jon: 271.0
Error for Kevin: 267.0
Error for Nate: 262.0


## Top 5 Misses by Nate


In [20]:
temp = clean.copy()
temp = temp.sort_values(by='Nate_error', ascending=False)
temp[['Name', 'Tier','Nate']][0:5]


Name Tier Nate
189          Zoetic Glyph   A+   C+
56          Sunfire Torch   C+    G
33   Child of the Volcano   D-   C+
158    Confounding Riddle    C   A-
28         Ancestors' Aid    C    G

## Top 5 Misses by Kevin


In [21]:
temp = clean.copy()
temp = temp.sort_values(by='Kevin_error', ascending=False)
temp[['Name', 'Tier','Kevin']][0:5]

Name Tier Kevin
165  Clay-Fired Bricks   B+    D+
41       Etali's Favor    B     D
166   Inverted Iceberg   C+    D-
28      Ancestors' Aid    C     G
188  Waylaying Pirates    C     G

## Top 5 Misses by Jon

In [22]:
temp = clean.copy()
temp = temp.sort_values(by='Jon_error', ascending=False)
temp[['Name', 'Tier','Jon']][0:5]

Name Tier Jon
189            Zoetic Glyph   A+  C+
41            Etali's Favor    B   D
2    Deep Goblin Skulltaker   D-  C+
143                 Petrify    B  D+
28           Ancestors' Aid    C   G

# Identifying the Top Cards 

In [23]:
clean = clean.copy()
clean = clean.sort_values(by='Tier_num', ascending=False)
print('Error for Jon:', clean['Jon_error'][0:25].sum())
print('Error for Kevin:', clean['Kevin_error'][0:25].sum())
print('Error for Nate:', clean['Nate_error'][0:25].sum())

Error for Jon: 58.0
Error for Kevin: 64.0
Error for Nate: 61.0


## Identifying the Worst Cards 

In [24]:
clean = clean.copy()
clean = clean.sort_values(by='Tier_num', ascending=True)
print('Error for Jon:', clean['Jon_error'][0:25].sum())
print('Error for Kevin:', clean['Kevin_error'][0:25].sum())
print('Error for Nate:', clean['Nate_error'][0:25].sum())

Error for Jon: 47.0
Error for Kevin: 36.0
Error for Nate: 37.0


In [25]:
clean = clean.copy()
clean['Kevin_hot'] = abs(((clean['Nate_num'] + clean['Jon_num']) / 2) - clean['Kevin_num'])
clean['Jon_hot'] = abs(((clean['Kevin_num'] + clean['Nate_num']) / 2) - clean['Jon_num'])
clean['Nate_hot'] = abs(((clean['Kevin_num'] + clean['Jon_num']) / 2) - clean['Nate_num'])
clean

Name Tier Jon Kevin Nate  Tier_num  Jon_num  Kevin_num  \
197      Canonized in Blood    F  C-    D-    G       0.0      4.0        1.0   
70        Disturbed Slumber    F   G    D-    G       0.0      0.0        1.0   
105  Might of the Ancestors    F   G     G    G       0.0      0.0        0.0   
93       Seeker of Sunlight   D-   G    C-   C-       1.0      0.0        4.0   
46            Hotfoot Gnome   D-  D+     D   D-       1.0      3.0        2.0   
..                      ...  ...  ..   ...  ...       ...      ...        ...   
165       Clay-Fired Bricks   B+  C-    D+   C-       9.0      4.0        3.0   
3           Deep-Cavern Bat   A-   B     B    B      10.0      8.0        8.0   
183        Staunch Crewmate   A-   B    C+   C+      10.0      8.0        6.0   
182          Spyglass Siren    A   B     B    B      11.0      8.0        8.0   
189            Zoetic Glyph   A+  C+     B   C+      12.0      6.0        8.0   

     Nate_num  Jon_error  Kevin_error  Nate_error  Kevin_hot  Jon_hot  \
197       0.0        4.0          1.0         0.0        1.0      3.5   
70        0.0        0.0          1.0         0.0        1.0      0.5   
105       0.0        0.0          0.0         0.0        0.0      0.0   
93        4.0        1.0          3.0         3.0        2.0      4.0   
46        1.0        2.0          1.0         0.0        0.0      1.5   
..        ...        ...          ...         ...        ...      ...   
165       4.0        5.0          6.0         5.0        1.0      0.5   
3         8.0        2.0          2.0         2.0        0.0      0.0   
183       6.0        2.0          4.0         4.0        1.0      2.0   
182       8.0        3.0          3.0         3.0        0.0      0.0   
189       6.0        6.0          4.0         6.0        2.0      1.0   

     Nate_hot  
197       2.5  
70        0.5  
105       0.0  
93        2.0  
46        1.5  
..        ...  
165       0.5  
3         0.0  
183       1.0  
182       0.0  
189       1.0  

[147 rows x 15 columns]

## Nate's Hottest Takes 


In [26]:
clean = clean.sort_values(by='Nate_hot', ascending=False)
clean[['Name', 'Tier', 'Nate', 'Kevin', 'Jon']][0:5]

Name Tier Nate Kevin Jon
57      Sunshot Militia    C   C-     D   G
197  Canonized in Blood    F    G    D-  C-
186    Waterlogged Hulk    D    D    D+  C+
93   Seeker of Sunlight   D-   C-    C-   G
143             Petrify    B   C+     C  D+

## Kevin's Hottest Takes 


In [27]:
clean = clean.sort_values(by='Kevin_hot', ascending=False)
clean[['Name', 'Tier', 'Kevin', 'Nate', 'Jon']][0:5]

Name Tier Kevin Nate Jon
26    Vito's Inquisitor   D-    D-   C-  C-
48    Panicked Altisaur   C-     D    C   C
92   Malamet War Scribe    C     C   C+   B
93   Seeker of Sunlight   D-    C-   C-   G
189        Zoetic Glyph   A+     B   C+  C+

## Jon's Hottest Takes 

In [28]:
clean = clean.sort_values(by='Jon_hot', ascending=False)
clean[['Name', 'Tier', 'Jon', 'Kevin', 'Nate']][0:5]

Name Tier Jon Kevin Nate
93   Seeker of Sunlight   D-   G    C-   C-
186    Waterlogged Hulk    D  C+    D+    D
197  Canonized in Blood    F  C-    D-    G
57      Sunshot Militia    C   G     D   C-
91   River Herald Guide    D  D+    C+   C+

### suggestions
- overall winner DONE
- biggest misses by person DONE
- how well did we identify top 25 cards 
- how well did we identify the bottom 25 cards 
- hot takes 
